# Data Manipulation with Pandas

Pandas supports 1-D (Series), 2-D (DataFrame), and 3-D (Panel) data structures.  Here we cover DataFrames because they most closely resemble the kind of data tables data scientists mostly look at.

The advantage of Pandas is that it stores the data together with its *metadata*.

The most often used meta data with Pandas are the **column names** and the **index**.


In [1]:
import pandas as pd
import numpy # for random number generation

In [2]:
df = pd.read_csv("assets/mammals.csv")

In [3]:
df

Legs Wings  Fur Feathers  Mammal
0     4    no  yes       no    True
1     2   yes   no      yes   False
2     4    no   no       no   False
3     4   yes  yes       no    True
4     3    no   no       no   False

# DataFrame Parts

A dataframe is composed of different parts that work together to give a coherent view of the data:

In [4]:
df.columns

Index(['Legs', 'Wings', 'Fur', 'Feathers', 'Mammal'], dtype='object')

In [5]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [6]:
df.values

array([[4, 'no', 'yes', 'no', True],
       [2, 'yes', 'no', 'yes', False],
       [4, 'no', 'no', 'no', False],
       [4, 'yes', 'yes', 'no', True],
       [3, 'no', 'no', 'no', False]], dtype=object)

We can change the parts of the data.  For example, we can create a new index for our dataframe:

In [7]:
df.index = ['Dog', 'Duck', 'Frog', 'Bat', 'Bar Stool']

In [8]:
df

Legs Wings  Fur Feathers  Mammal
Dog           4    no  yes       no    True
Duck          2   yes   no      yes   False
Frog          4    no   no       no   False
Bat           4   yes  yes       no    True
Bar Stool     3    no   no       no   False

# Indexing and Slicing

For array-style indexing Pandas  uses the **loc**, **iloc**, and **ix** indexers. 

Using the **iloc** indexer, we can index the underlying array as if it is a simple array using row and column integer values (hence the i in iloc). The DataFrame index and column labels are maintained in the result:

In [9]:
df

Legs Wings  Fur Feathers  Mammal
Dog           4    no  yes       no    True
Duck          2   yes   no      yes   False
Frog          4    no   no       no   False
Bat           4   yes  yes       no    True
Bar Stool     3    no   no       no   False

In [10]:
df.iloc[:2,1:4]

Wings  Fur Feathers
Dog     no  yes       no
Duck   yes   no      yes

Using the **loc** indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [11]:
df.loc[:'Duck','Wings':'Feathers']

Wings  Fur Feathers
Dog     no  yes       no
Duck   yes   no      yes

Notice that when slicing with an explicit index (i.e., data.loc['a':'c']), the final index is included in the slice, while when slicing with an implicit index (i.e., data.iloc[0:2]), the final index is excluded from the slice.

The indexer **ix** allows the mix of integer and explicit indexing.

# Data Access Patterns

We can use relational and boolean expressions when selecting data from a dataframe.

In order to see that we have to realize that there is another simple way to select frame columns:

In [12]:
df[['Wings', 'Mammal']]

Wings  Mammal
Dog          no    True
Duck        yes   False
Frog         no   False
Bat         yes    True
Bar Stool    no   False

Relational Operators:

In [13]:
df[df.Wings == 'yes']

Legs Wings  Fur Feathers  Mammal
Duck     2   yes   no      yes   False
Bat      4   yes  yes       no    True

In [14]:
df[(df.Wings == 'yes') & (df.Fur == 'yes')]

Legs Wings  Fur Feathers  Mammal
Bat     4   yes  yes       no    True

In [15]:
df[(df.Wings == 'yes') & (df.Fur == 'yes')].Mammal

Bat    True
Name: Mammal, dtype: bool

# Filtering

The previous operations are just a special case of *filtering*: boolean indexing using a Boolean vector.

In [16]:
df.Mammal

Dog           True
Duck         False
Frog         False
Bat           True
Bar Stool    False
Name: Mammal, dtype: bool

In [17]:
df.Mammal.dtype

dtype('bool')

In [18]:
df[df.Mammal]

Legs Wings  Fur Feathers  Mammal
Dog     4    no  yes       no    True
Bat     4   yes  yes       no    True

# Missing Data
* Pandas flags missing values with NaN (not a number).
* In most cases, any computations applied to a dataframe with NaNs will ignore the NaNs
* However, it is still a good idea to clean up the dataframe
* In general we have two options to deal with missing data:
 * Either drop the row or columns that has NaNs
 * Or try to substitute a reasonable value for the NaN
 

Generate a dataset with NaNs

In [19]:
df = pd.DataFrame(numpy.random.randn(4, 3), index=['a', 'c', 'd', 'e'], 
                columns=['one', 'two', 'three'])
df

one       two     three
a -0.814529 -0.887127  0.423467
c  0.058174 -1.041722  0.496293
d -0.070479 -1.556315  0.246489
e -0.727232  0.029788  1.456365

Generating NaNs

In [20]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e']) # creating new row 'b' with no values
df2

one       two     three
a -0.814529 -0.887127  0.423467
b       NaN       NaN       NaN
c  0.058174 -1.041722  0.496293
d -0.070479 -1.556315  0.246489
e -0.727232  0.029788  1.456365

In [21]:
# find the places where the NaNs are
df2.isnull()

one    two  three
a  False  False  False
b   True   True   True
c  False  False  False
d  False  False  False
e  False  False  False

In [22]:
# look at the values of the isnull dataframe
df2.isnull().values

array([[False, False, False],
       [ True,  True,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False]], dtype=bool)

In [23]:
# find out how many values are missing
# NOTE: sum treats 'True' as 1 and 'False' as 0 
df2.isnull().values.sum()

3

In [24]:
# drop rows that have NaNs
df2.dropna(how='any',axis=0)

one       two     three
a -0.814529 -0.887127  0.423467
c  0.058174 -1.041722  0.496293
d -0.070479 -1.556315  0.246489
e -0.727232  0.029788  1.456365

In [25]:
# dropping columns that have NaNs
# NOTE: this is NOT always a good idea -- empty dataframe!
df2.dropna(how='any',axis=1)

Empty DataFrame
Columns: []
Index: [a, b, c, d, e]

# Broadcasting

Binary arithmetic operators are applied element by element to dataframes assuming equal sized dataframes.

Broadcasting refers to the fact that Python will reuse elements of the smaller dataframe or will reuse a scalar in order to complete the binary operation.


In [26]:
df = pd.DataFrame([[1,2],[3,4]])
df

0  1
0  1  2
1  3  4

In [27]:
# element by element operation
df + df

0  1
0  2  4
1  6  8

In [28]:
# broadcasting the smaller vector
# NOTE: each element of the vector is applied to 
#       a column in the dataframe
df + [10, 20]

0   1
0  11  22
1  13  24

In [29]:
# broadcasting a scalar
# NOTE: the scalar is applied to ALL elements
#       of the dataframe
df + 10

0   1
0  11  12
1  13  14

In [30]:
# we can now say things like this
df + df == 2*df

0     1
0  True  True
1  True  True

# Reading

* 2 [The Basics of NumPy Arrays](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html)
* 3.2 [Data Indexing and Selection](https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html)
* 3.3 [Operating on Data in Pandas](https://jakevdp.github.io/PythonDataScienceHandbook/03.03-operations-in-pandas.html)
* 3.4 [Handling Missing Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html)
* 3.6 [Combining Datasets: Concat and Append](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html)
* 3.7 [Combining Datasets: Merge and Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)
* 3.8 [Aggregation and Grouping](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)